In [1]:
import os 
import pandas as pd
from nilearn import datasets, plotting
from bwas_wrapper.wrapper import wrapper

In [2]:
from bwas_wrapper.BWAS import BWAS_cpu

In [3]:
os.path.dirname(BWAS_cpu.__file__)

'/home/pbellec/git/bwas_wrapper/bwas_wrapper/BWAS'

Let's start by downloading a few subjects from the ADHD200 dataset.

In [4]:
n_subjects = 30
adhd_dataset = datasets.fetch_adhd(n_subjects)
epi_filename = adhd_dataset.func
# Also load a probabilistic grey matter segmentation
mni = datasets.fetch_icbm152_2009()

/home/pbellec/env/simonsvip/lib/python3.7/site-packages/nilearn/datasets/func.py:516: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


Let's load the phenotypic information in a pandas DataFrame and check what variables there are:

In [5]:
data = pd.DataFrame(adhd_dataset.phenotypic)
data.columns

Index(['f0', 'Subject', 'RestScan', 'MeanFD', 'NumFD_greater_than_020',
       'rootMeanSquareFD', 'FDquartiletop14thFD', 'PercentFD_greater_than_020',
       'MeanDVARS', 'MeanFD_Jenkinson', 'site', 'sibling_id', 'data_set',
       'age', 'sex', 'handedness', 'full_2_iq', 'full_4_iq', 'viq', 'piq',
       'iq_measure', 'tdc', 'adhd', 'adhd_inattentive', 'adhd_combined',
       'adhd_subthreshold', 'diagnosis_using_cdis', 'notes', 'sess_1_anat_2',
       'oppositional', 'cog_inatt', 'hyperac', 'anxious_shy', 'perfectionism',
       'social_problems', 'psychosomatic', 'conn_adhd', 'restless_impulsive',
       'emot_lability', 'conn_gi_tot', 'dsm_iv_inatt', 'dsm_iv_h_i',
       'dsm_iv_tot', 'study', 'sess_1_rest_1', 'sess_1_rest_1_eyes',
       'sess_1_rest_2', 'sess_1_rest_2_eyes', 'sess_1_rest_3',
       'sess_1_rest_3_eyes', 'sess_1_rest_4', 'sess_1_rest_4_eyes',
       'sess_1_rest_5', 'sess_1_rest_5_eyes', 'sess_1_rest_6',
       'sess_1_rest_6_eyes', 'sess_1_anat_1', 'sess_1_which

We need to select the variables to include in our analyis, and also have a column with the subject IDs:

In [6]:
model = data[['Subject', 'adhd', 'MeanFD', 'age']]

let's add sex as well, but this variable is categorical. We need to convert it into a dummy variable first.

In [7]:
sex = pd.get_dummies(data['sex'], prefix='sex', drop_first=True) # drop_first is applied because the model will include an intercept
model = pd.concat([model, sex], axis=1)

we can select subjects too. For example, let's filter subjects with excessive motion.

In [8]:
model = model[model['MeanFD']<0.1]

Let's add an intercept to the regression model:

In [9]:
model.insert(0, column='Intercept', value=1)

Finally, we need to add a dictionary with the file of the preprocessed data for each subject. The IDs of the subject need to correspond to the subject IDs in the data frame. 

In [10]:
# By checking `adhd_dataset.func`, we can easily pick up the naming patterns of the functional files
# '/home/pbellec/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz'
path_data = os.path.join( os.getenv('HOME'), 'nilearn_data', 'adhd', 'data')
files = {}
for subject in model['Subject']:
    subject_prefix = f'{subject :7}'.replace(' ','0') 
    file = f'{subject_prefix}_rest_tshift_RPI_voreg_mni.nii.gz'
    files[subject] = os.path.join(path_data, f'{subject_prefix}', file)
files

{10042: '/home/pbellec/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz',
 10064: '/home/pbellec/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz',
 10128: '/home/pbellec/nilearn_data/adhd/data/0010128/0010128_rest_tshift_RPI_voreg_mni.nii.gz',
 21019: '/home/pbellec/nilearn_data/adhd/data/0021019/0021019_rest_tshift_RPI_voreg_mni.nii.gz',
 23008: '/home/pbellec/nilearn_data/adhd/data/0023008/0023008_rest_tshift_RPI_voreg_mni.nii.gz',
 23012: '/home/pbellec/nilearn_data/adhd/data/0023012/0023012_rest_tshift_RPI_voreg_mni.nii.gz',
 27018: '/home/pbellec/nilearn_data/adhd/data/0027018/0027018_rest_tshift_RPI_voreg_mni.nii.gz',
 27034: '/home/pbellec/nilearn_data/adhd/data/0027034/0027034_rest_tshift_RPI_voreg_mni.nii.gz',
 27037: '/home/pbellec/nilearn_data/adhd/data/0027037/0027037_rest_tshift_RPI_voreg_mni.nii.gz',
 1019436: '/home/pbellec/nilearn_data/adhd/data/1019436/1019436_rest_tshift_RPI_voreg_mni.nii.gz',
 1206380: '/home/pbellec/nil

In [11]:
wrapper(model, files, result_dir='/tmp/test/', subject_id="Subject", interest="adhd")

Creating sym links in the following temporary folder: /tmp/tmprrdbty95
linking /home/pbellec/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000000.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000001.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0010128/0010128_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000002.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0021019/0021019_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000003.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0023008/0023008_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000004.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0023012/0023012_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty95/000000000005.nii.gz
linking /home/pbellec/nilearn_data/adhd/data/0027018/0027018_rest_tshift_RPI_voreg_mni.nii.gz with /tmp/tmprrdbty

NameError: name 'np' is not defined

In [ ]:
plotting.view_img('/tmp/tmpx9iwgjyg/mask.nii.gz')